In [3]:
import pandas as pd
import numpy as np
from pydataset import data

In [20]:
import pyspark
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean, concat_ws, lit
from pyspark.sql.functions import regexp_extract, regexp_replace
spark = pyspark.sql.SparkSession.builder.getOrCreate()

# 1
Create a spark data frame that contains your favorite programming languages.

    The name of the column should be language
    View the schema of the dataframe
    Output the shape of the dataframe
    Show the first 5 records in the dataframe

In [ ]:
#df = spark.createDataFrame(pd.DataFrame())

# 2
Load the mpg dataset as a spark dataframe.

    Create 1 column of output that contains a message like the one below:


    The 1999 audi a4 has a 4 cylinder engine.
    For each vehicle.

   

In [4]:
mpg = spark.createDataFrame(data("mpg"))

In [168]:
mpg.select(concat(
            lit("The "), 
            mpg.year, 
            lit(" "), 
            mpg.manufacturer, 
            lit(" "), 
            mpg.model, 
            lit(" has a "), 
            mpg.cyl, 
            lit(" cylinder engine.")).alias("The Vehicle's Description")).show(5,truncate=False)

+-----------------------------------------+
|The Vehicle's Description                |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



    Transform the trans column so that it only contains either manual or auto.

In [28]:
mpg.select(
    regexp_extract("trans", r"(auto|manual)", 1).alias("trans_type")).show(truncate=False)


+----------+
|trans_type|
+----------+
|auto      |
|manual    |
|manual    |
|auto      |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|auto      |
|manual    |
|auto      |
|manual    |
|auto      |
|auto      |
|auto      |
|auto      |
|auto      |
+----------+
only showing top 20 rows



In [19]:
mpg.select(mpg.trans).show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



# 3
Load the tips dataset as a spark dataframe.

    What percentage of observations are smokers?

In [29]:
tips = spark.createDataFrame(data("tips"))

In [32]:
tips.where(tips["smoker"] == "Yes").show(5)

+----------+----+----+------+---+------+----+
|total_bill| tip| sex|smoker|day|  time|size|
+----------+----+----+------+---+------+----+
|     38.01| 3.0|Male|   Yes|Sat|Dinner|   4|
|     11.24|1.76|Male|   Yes|Sat|Dinner|   2|
|     20.29|3.21|Male|   Yes|Sat|Dinner|   2|
|     13.81| 2.0|Male|   Yes|Sat|Dinner|   2|
|     11.02|1.98|Male|   Yes|Sat|Dinner|   2|
+----------+----+----+------+---+------+----+
only showing top 5 rows



In [169]:
tips.count()

244

In [42]:
tips.groupBy(tips.smoker).agg((count(tips.smoker)/244).alias("Percentage of 244 Customers")).show()

+------+---------------------------+
|smoker|Percentage of 244 Customers|
+------+---------------------------+
|    No|         0.6188524590163934|
|   Yes|        0.38114754098360654|
+------+---------------------------+



In [170]:
tips.groupBy(tips.smoker).agg((count(tips.smoker)/tips.count()).alias("Percentage of 244 Customers")).show()

+------+---------------------------+
|smoker|Percentage of 244 Customers|
+------+---------------------------+
|    No|         0.6188524590163934|
|   Yes|        0.38114754098360654|
+------+---------------------------+



In [172]:
#The select statement chooses from the groupBy.count table made in the first two functions.
#The glob star selects all the rows. Adds a new one that is the count column divided by the value created by
#tips.count(). This new row is then aliased.
tips.groupBy('smoker').count().select('*', (col('count')/tips.count()).alias('percent')).show()

+------+-----+-------------------+
|smoker|count|            percent|
+------+-----+-------------------+
|    No|  151| 0.6188524590163934|
|   Yes|   93|0.38114754098360654|
+------+-----+-------------------+



    Create a column that contains the tip percentage

In [44]:
tips.select((tips.tip/tips.total_bill).alias("Tip as Percentge of Bill")).show(5)

+------------------------+
|Tip as Percentge of Bill|
+------------------------+
|     0.05944673337257211|
|     0.16054158607350097|
|     0.16658733936220846|
|      0.1397804054054054|
|     0.14680764538430255|
+------------------------+
only showing top 5 rows



In [45]:
tip_percent = tips.select((tips.tip/tips.total_bill).alias("Tip as Percentge of Bill"))

    Calculate the average tip percentage for each combination of sex and smoker.

In [49]:
sexy_smokers = tips.groupby("sex", "smoker")

In [55]:
sexy_smokers.agg(avg('tip')/avg('total_bill')).show()

+------+------+----------------------------+
|   sex|smoker|(avg(tip) / avg(total_bill))|
+------+------+----------------------------+
|  Male|    No|          0.1573121500195338|
|  Male|   Yes|         0.13691878510474398|
|Female|    No|         0.15318918255461905|
|Female|   Yes|         0.16306234935189712|
+------+------+----------------------------+



# 4
Use the seattle weather dataset referenced in the lesson to answer the questions below.

In [57]:
from vega_datasets import data

weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)

    Convert the temperatures to farenheight.

In [59]:
weather.describe().show()

+-------+----------+-----------------+-----------------+-----------------+------------------+-------+
|summary|      date|    precipitation|         temp_max|         temp_min|              wind|weather|
+-------+----------+-----------------+-----------------+-----------------+------------------+-------+
|  count|      1461|             1461|             1461|             1461|              1461|   1461|
|   mean|      null|3.029431895961668|16.43908281998631| 8.23477070499658|3.2411362080766604|   null|
| stddev|      null|6.680194322314739|7.349758097360176|5.023004179961265|1.4378250588746198|   null|
|    min|2012-01-01|              0.0|             -1.6|             -7.1|               0.4|drizzle|
|    max|2015-12-31|             55.9|             35.6|             18.3|               9.5|    sun|
+-------+----------+-----------------+-----------------+-----------------+------------------+-------+



In [63]:
weather.select((weather.temp_max*9/5+32).alias("Max_Fahrenheit"),
              (weather.temp_min*9/5+32).alias("Min_Fahrenheit")).show(5)

+--------------+--------------+
|Max_Fahrenheit|Min_Fahrenheit|
+--------------+--------------+
|         55.04|          41.0|
|         51.08|         37.04|
|         53.06|         44.96|
|         53.96|         42.08|
|         48.02|         37.04|
+--------------+--------------+
only showing top 5 rows



    Which month has the most rain, on average?

In [72]:
from pyspark.sql.functions import month, year, quarter, col

In [75]:
monthtly_total_rain = weather.withColumn("month", month("date")).groupBy("month").agg(sum("precipitation").alias("monthly_total_rainfall"))

In [74]:
monthtly_total_rain.sort(col("monthly_total_rainfall").desc()).show(1)

+-----+----------------------+
|month|monthly_total_rainfall|
+-----+----------------------+
|   11|                 642.5|
+-----+----------------------+
only showing top 1 row



In [1]:
monthtly_total_rain.sort(col("monthly_total_rainfall").desc()).first()

NameError: name 'monthtly_total_rain' is not defined

In [64]:
weather.dtypes

[('date', 'string'),
 ('precipitation', 'double'),
 ('temp_max', 'double'),
 ('temp_min', 'double'),
 ('wind', 'double'),
 ('weather', 'string')]

    Which year was the windiest?

In [76]:
windy_years = weather.withColumn("year", year("date")).groupBy("year").agg(sum("wind").alias("annual_wind"))

In [80]:
windy_years.sort(col('annual_wind').desc()).show(1)

+----+------------------+
|year|       annual_wind|
+----+------------------+
|2012|1244.6999999999998|
+----+------------------+
only showing top 1 row



    What is the most frequent type of weather in January?

In [92]:
with_month_col = weather.withColumn("month", month("date"))
#.where(weather["month"] == 1)

In [97]:
with_month_col.where(with_month_col.month == 1).crosstab("month", "weather").show()

+-------------+-------+---+----+----+---+
|month_weather|drizzle|fog|rain|snow|sun|
+-------------+-------+---+----+----+---+
|            1|     10| 38|  35|   8| 33|
+-------------+-------+---+----+----+---+



In [95]:
with_month_col.where(with_month_col["month"] == 1).show()

+----------+-------------+--------+--------+----+-------+-----+
|      date|precipitation|temp_max|temp_min|wind|weather|month|
+----------+-------------+--------+--------+----+-------+-----+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|    1|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|    1|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|    1|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|    1|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|    1|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|    1|
|2012-01-07|          0.0|     7.2|     2.8| 2.3|   rain|    1|
|2012-01-08|          0.0|    10.0|     2.8| 2.0|    sun|    1|
|2012-01-09|          4.3|     9.4|     5.0| 3.4|   rain|    1|
|2012-01-10|          1.0|     6.1|     0.6| 3.4|   rain|    1|
|2012-01-11|          0.0|     6.1|    -1.1| 5.1|    sun|    1|
|2012-01-12|          0.0|     6.1|    -1.7| 1.9|    sun|    1|
|2012-01-13|          0.0|     5.0|    -

    What is the average high and low tempurature on sunny days in July in 2013 and 2014?

In [100]:
just_july = with_month_col.filter(with_month_col.month == 7)

In [102]:
july_years = just_july.withColumn("year", year("date"))

In [106]:
july2013_14 = july_years.filter((july_years.year == 2013)|(july_years.year == 2014))

In [115]:
july2013_14.filter(july2013_14.weather == 'sun').groupBy('weather').agg(avg('temp_max'), avg('temp_min')).show()

+-------+------------------+-----------------+
|weather|     avg(temp_max)|    avg(temp_min)|
+-------+------------------+-----------------+
|    sun|26.828846153846158|14.18269230769231|
+-------+------------------+-----------------+



In [111]:
sunny_july_2013_14 = july2013_14.filter(july2013_14.weather == 'sun')

In [ ]:
sunny_july_2013_14.

In [ ]:
#mpg.filter(mpg.cyl == 4).where(mpg["class"] == "subcompact").show()

    What percentage of days were rainy in q3 of 2015?

In [130]:
weather_2015 = weather_years.filter(weather_years.year == '2015') 

In [135]:
weather_2015Q3 = weather_2015.filter((weather.date > '2015-07-01') & (weather.date < '2015-09-30'))

In [138]:
weather_2015Q3.groupBy('weather').agg(count('weather')).show()

+-------+--------------+
|weather|count(weather)|
+-------+--------------+
|    fog|            20|
|drizzle|             5|
|   rain|             2|
|    sun|            63|
+-------+--------------+



In [166]:
weather_2015Q3.groupBy('weather').agg((count('weather')/q3size)).show()

+-------+---------------------+
|weather|(count(weather) / 90)|
+-------+---------------------+
|    fog|   0.2222222222222222|
|drizzle|  0.05555555555555555|
|   rain| 0.022222222222222223|
|    sun|                  0.7|
+-------+---------------------+



In [165]:
q3size = weather_2015Q3.count()

In [149]:
weather_2015Q3.rollup('weather').count().show()

+-------+-----+
|weather|count|
+-------+-----+
|   null|   90|
|    fog|   20|
|    sun|   63|
|   rain|    2|
|drizzle|    5|
+-------+-----+



    For each year, find what percentage of days it rained (had non-zero precipitation).

In [155]:
weather_years.groupBy('year').agg(sum('precipitation')).show()

+----+------------------+
|year|sum(precipitation)|
+----+------------------+
|2015|1139.1999999999996|
|2013| 827.9999999999995|
|2014| 1232.799999999999|
|2012|1225.9999999999989|
+----+------------------+



In [157]:
no_rain = weather_years.filter(weather_years.precipitation == 0)

In [164]:
no_rain.groupBy('year').agg((count('year')/365).alias("% of Year with 0 Precipitation")).show()                            
#tips.groupBy(tips.smoker).agg((count(tips.smoker)/244).alias("Percentage of 244 Customers")).show()

+----+------------------------------+
|year|% of Year with 0 Precipitation|
+----+------------------------------+
|2015|            0.6054794520547945|
|2013|            0.5835616438356165|
|2014|             0.589041095890411|
|2012|            0.5178082191780822|
+----+------------------------------+

